Given code: 
1. import packages
2. load image pairs img1 & img2

In [46]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load image pairs - you can change file names
image1 = cv2.imread("IMG_7577.png")
image2 = cv2.imread("IMG_7578.png")

Problem 1: [code by yourself]

Define a function `get_transform_from_keypoints()` that computes the 3x3 homogeneous transform H from keypoint matches with the following input parameters and return variables:
* img_src (input): image that is warped to be aligned to img_dst 
* img_dst (input): reference image to align img_src
* H (output/return): computed linear transform matrix
* kpts_src (output/return): computed keypoints for img_src
* dscrpt_src (output/return): computed descriptors for img_src
* kpts_dst (output/return): computed keypoints for img_dst
* dscrpt_dst (output/return): computed descriptors for img_dst
* matches (output/return): keypoint matches determined from SIFT

When computing the linear transform, follow this proceses:
* detect SIFT keypoints and compute SIFT descriptors
* find the linear transform matrix H using the matched keypoints

In [54]:
def get_transform_from_keypoints(img_src, img_dst):
    img_src_gray = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)
    img_dst_gray = cv2.cvtColor(img_dst, cv2.COLOR_BGR2GRAY)

    # Create a SIFT object and detect keypoints and descriptors for each image
    sift = cv2.SIFT_create()
    kpts_src, dscrpt_src = sift.detectAndCompute(img_src_gray, None)
    kpts_dst, dscrpt_dst = sift.detectAndCompute(img_dst_gray, None)
    #print(dscrpt_src, dscrpt_dst)

    # Match the descriptors using Brute-Force matcher
    bf = cv2.BFMatcher()
    matches = bf.match(dscrpt_src, dscrpt_dst)

    # Sort the matches in the order of their distances
    matches = sorted(matches, key=lambda x: x.distance)

    # Find the homography matrix using the matched keypoints
    src_pts = np.float32([kpts_src[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kpts_dst[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 2.0)
    matches = [matches[i] for i in range(len(matches)) if mask[i]]

    # Return outputs
    return H, kpts_src, dscrpt_src, kpts_dst, dscrpt_dst, matches

Problem 2: [Code by yourself]

Define a function named `stitch_image()` that generates a stitched image from img_ref, img_align, and T, with the following input parameters and return variables:
* img_src (input): image that is warped to be aligned to img_dst 
* img_dst (input): reference image to align img_src
* H (input): computed linear transform matrix
* stitched_image (output): the stitched image

This function should include the following processes:
* Compute the size of the output stitched image and the offset that ensures all pixels from both images are included in the stitched image
* Modify H to account for the image offset
* Warp the first src image into the second image
* Blend in the second dst image

In [62]:
def get_stitched_image(img_src, img_dst, H):
    # Compute the size of the output stitched image
    rows_src, cols_src = img_src.shape[:2]
    rows_dst, cols_dst = img_dst.shape[:2]
  
    # 첫 번째 이미지 꼭지점 좌표 생성
    corners_src = np.float32([[0, 0], [0, rows_src], [cols_src, rows_src], [cols_src, 0]]).reshape(-1, 1, 2)
  
    # H변환행렬을 통한 1st 이미지의 꼭지점좌표를 2nd 이미지의 좌표공간으로 변환
    corners_dst = cv2.perspectiveTransform(corners_src, H)
    
    # 2nd 이미지의 꼭지점 좌표와 1st 이미지의 꼭지점 좌표 합치기
    corners = np.concatenate((corners_dst, corners_src), axis=0)
    
    # 변환된 자표들 중 최대/최소 x, y좌표 구하기
    [x_min, y_min] = np.int32(corners.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(corners.max(axis=0).ravel() + 0.5)
    
    # 음수값을 보정하기 위한 offset 설정
    offset_x = -x_min if x_min < 0 else 0
    offset_y = -y_min if y_min < 0 else 0
    
    # 출력 이미지 너비와 높이 계산
    dst_width = max(x_max, cols_dst) - min(x_min, 0)
    dst_height = max(y_max, rows_dst) - min(y_min, 0)

    # Modify H to account for the image offset - 변환행렬 오프셋 고려
    H_modified = np.array([[1, 0, offset_x], [0, 1, offset_y], [0, 0, 1]]) @ H
  
    # Warp the first image to the perspective of the second image
    # -> img_src를 2nd 이미지의 원근 변환에 맞게 변환
    warped_img = cv2.warpPerspective(img_src, H_modified, (dst_width, dst_height))

    # Alpha blending
    alpha = 0  # 가중치 값 (조절 가능)
    stitched_image = np.zeros((dst_height, dst_width, 3), dtype=np.uint8)
    stitched_image[offset_y:rows_dst + offset_y, offset_x:cols_dst + offset_x] = img_dst

    # warped_img 그레이 스케일로 변환 & 임계값 기준으로 이진화하여 mask_warped 생성
    mask_warped = cv2.cvtColor(warped_img, cv2.COLOR_BGR2GRAY)
    ret, mask_warped = cv2.threshold(mask_warped, 1, 255, cv2.THRESH_BINARY)

    # mask_dst 생성
    mask_dst = cv2.bitwise_not(mask_warped)

    # 위에서 생성된 두 마스크 조합
    mask_combined = cv2.bitwise_and(mask_warped, mask_dst)

    # 각 이미지에 마스크를 적용하여 일부 영역을 검은색으로 만듬
    stitched_image = cv2.bitwise_and(stitched_image, cv2.cvtColor(mask_dst, cv2.COLOR_GRAY2BGR))
    warped_img = cv2.bitwise_and(warped_img, cv2.cvtColor(mask_warped, cv2.COLOR_GRAY2BGR))
    
    # stiched_image와 warped image를 알파 블렌딩 수행
    stitched_image = cv2.add(stitched_image, warped_img)

    # return output
    return stitched_image


Given code: 
3. Call function `get_transform_from_keypoints` - detect and match keypoints and compute transform
4. Draw the matches on a new image to check validity of matched keypoints
5. Call function `get_stitched_image` - create stitched image and save

In [64]:
# 3. Detect and match keypoints and compute transform
H, kpts_src, _, kpts_dst, _, matches = get_transform_from_keypoints(image1, image2)

# 4. Draw the matches on a new image to check validity of matched keypoints
match_image = cv2.drawMatches(image1, kpts_src, image2, kpts_dst, matches, None)
cv2.imwrite('matches.png', match_image)

# 3. Create stitched image and save
stitched_image = get_stitched_image(image1, image2, H)
cv2.imwrite('stitched.png', stitched_image)

True